In [1]:
import numpy as np
import pandas as pd

from typing import List, Tuple
from numpy.typing import ArrayLike


# The change problem

## La solución de programación dinámica

* Supongamos que enumeramos las monedas por valores crecientes $v_1, \ldots, v_i, \ldots v_N$. Queremos cambiar una cantidad $C$.
* Sea $n(i, c)$ el número mínimo de monedas (o cambio óptimo) que se necesitan para devolver el cambio de una cantidad $0 \leq c \leq C$ usando tan sólo las primeras $i$ monedas de la enumeración. El cambio óptimo **solo** puede ser una de las siguientes dos posibilidades:
 * **O bien** el cambio óptimo para dicha cota $c$ seleccionando monedas incluidas entre las $i-1$ primeras monedas de la enumeración. Es decir la moneda $i$ **no** entra en el cambio y, por tanto  $n(i, c) = n(i-1, c)$ 
 * **O bien** el cambio óptima considerando las primeras $i$ monedas para la cota $c-v_i$  (esto es, sustrayendo de la cota $c$ el valor de una de las monedas $i$ con valor $v_i$) **mas** uno (al tener que añadirse la moneda de valor $v_i$ que previamente habíamos sustraído)
 $$ 
 n(i, c) = 1 + n(i, c-v_i) 
 $$ 
 
* De las dos posibilidades se elige la de **menor** valor:
 
 \begin{equation}
 n(i, c) = \text{min} \big(  n(i-1, c), 1 + n(i, c-v_i) \big)   \; \; \; 1 < i \leq N, \; 1 \leq c \leq C
 \end{equation}
 
* Obviamente $n(i, 0) = 0$ para todo $i\leq N$ y $n(1, c)=c$ para todo $c\leq C$. El resto de los valores de la matriz $n(i,c)$ *se llenan* de **abajo hacia arriba**, *incrementando* el índice de las filas, y de izquierda a derecha, *incrementando* el valor de las columnas.

* El cambio óptimo será $n(N, C)$, *esquina inferior-derecha* de la matriz.

In [6]:
def minimum_change_dp (v:List, change:int) -> ArrayLike:
    '''v: sorted coins values'''
    
    # n(i, c): Total returned coins using the first i coins (rows) for c change
    n = np.zeros((len(v), change+1), dtype=np.int32)
    n[0, :] = np.arange (0, change+1)
    
    # Bottom up  
    for i in np.arange(1, len(v)):
        for c in np.arange (1, change+1):
            #sub-optimal structure
            if v[i] <= c :
                n[i, c] = min ((n[i-1, c], 1 + n[ i, c-v[i]] ))
            else :
                n[i, c] =  n[i-1, c]
    return n

#----------- Driver program

# sorted coins values
coins_value = [1, 3, 4, 5, 7]

# quantity to change
change = 10

n = minimum_change_dp (coins_value, change)

print(f'Matrix change n:\n{n}')
print (f'Change of {change}, total returned coins: {n[-1,-1]}') 


Matrix change n:
[[ 0  1  2  3  4  5  6  7  8  9 10]
 [ 0  1  2  1  2  3  2  3  4  3  4]
 [ 0  1  2  1  1  2  2  2  2  3  3]
 [ 0  1  2  1  1  1  2  2  2  2  2]
 [ 0  1  2  1  1  1  2  1  2  2  2]]
Change of 10, total returned coins: 2


In [7]:
### -------  Printing the n matrix as the slides --------
from IPython.display import display, HTML

# create a Pandas DateFrame
df = pd.DataFrame(n, index=coins_value)

# from DataFrame to html code 
html = df.to_html()

# display the n-matrix nice
display(HTML(html))

,0,1,2,3,4,5,6,7,8,9,10
1,0,1,2,3,4,5,6,7,8,9,10
3,0,1,2,1,2,3,2,3,4,3,4
4,0,1,2,1,1,2,2,2,2,3,3
5,0,1,2,1,1,1,2,2,2,2,2
7,0,1,2,1,1,1,2,1,2,2,2


**Corrección del algoritmo codicioso**

**Coste del algoritmo DP**


# The Knapsack 0/1 problem

## Introducción

* Recordad que tenemos $N$ elementos con pesos enteros $w_i$ y valores $v_i$ y una mochila que soporta un peso máximo $W$.

* Queremos una selección de elementos $i_1, \ldots, i_k$, tal que la suma de sus pesos **no** sea superior a $W$ y su valor conjunto sea máximo. Es decir,
$\sum_{j=1}^k w_{i_j} \leq W$ y $\sum_{j=1}^k v_{i_j}$ sea máxima. 

* Matemáticamente, queremos resolver un **Problema de Optimización** con **restricciones**
$$
\text{max} \left( \sum_1^N v_i x_i \right) \; \; \; \text{sujeto a} \; \; \;   \sum_1^N v_i x_i \leq W \; \; \; \text{y} \; \; \; x_i \in [0, 1] 
$$
* El nombre $0-1$ proviene de la restricción $x_i \in [0, 1] $
* Ya vimos que el problema tiene una solución natural codiciosa, que no es correcta.

## La solución de programación dinámica

* Supongamos que ordenamos los elementos por valores crecientes de sus pesos $w_1, \ldots, w_i, \ldots w_N$. 
* Sea $V(i, w)$ el valor óptimo de la mochila cuando **solo** se puedan seleccionar elementos incluidos en los $i$ primeros elementos de la enumeración y cuando se considere un cota  $0 \leq w \leq W$.

* El valor óptimo $V(i, w)$  **solo puede ser** uno de los dos siguientes: 
 * La mochila óptima **no incluye** al elemento $i$, último elemento de los que se pueden seleccionar. Es decir la mochila tan solo incluye elementos extraídos de los $(i-1)$ primeros elementos. En este caso, el valor de la mochila óptima $V(i, w)$ será el mismo que el de una mochila óptima formada con elementos seleccionados entre  los $i-1$ primeros elementos y la misma cota $w$ 
 
 $$V(i, w)= V(i-1, w)$$ 
 
 * **O bien** el elemento $i$ sí está en la selección óptima de los elementos incluidos en la mochila. Por tanto, el valor de la mochila será **necesariamente**  
 $$V(i,w) = v_i + V (i-1, w-w_i)$$
 Es decir, el valor de la mochila óptima será el de la mochila óptima considerando elementos de la enumeración $i-1$ y cota $w-w_i$, $V(i-1, w-w_i)$, añadiendo el valor aportado por el elemento $i$, $v_i$.

* De las dos posibilidades se elige la de **mayor** valor. Obtenemos así la ley de recurrencia
 $$
 V(i, j) = \text{max} \left( V(i-1, j-1), v_i + V(i-1, w - w_i) \right)
 $$
 
* Como condiciones de contorno  


\begin{align}
V(0,j) & =
\begin{cases}
0  & \text{si} \; \;  w_j > W  \\
w_0  & \text{en cualquier otro caso}  
\end{cases} \\
V(i, 0) &= 0
\end{align}


In [8]:

def maximum_gain (w:List, v:List, weight:int) -> ArrayLike:
    ''' Important w must be sorted'''
    
    V = np.zeros ((len(w), weight+1), dtype=np.int32)

    V[0,:] = [ 0 if i < w[0] else  v[0]  for i in np.arange(0, weight+1)]
    
    for i in  np.arange(1, len(w)):
        for j in np.arange(0, weight+1):
            if w[i]<= j:
                V[i, j] = max ( V[i-1, j], v[i] + V[i-1, j - w[i]] )
            else:
                V[i, j] =  V[i-1, j]
                
    return V

#---- Driver program-----

# input data
maximum_weight_kn = 13

items_weight = [4, 4, 5]
items_value = [10, 11, 15]

W = maximum_gain (items_weight, items_value, maximum_weight_kn)

print(W)
print(f'Optimum gain: {W[-1, -1]} for knapsack with maximum weight restriction: {maximum_weight_kn}')


[[ 0  0  0  0 10 10 10 10 10 10 10 10 10 10]
 [ 0  0  0  0 11 11 11 11 21 21 21 21 21 21]
 [ 0  0  0  0 11 15 15 15 21 26 26 26 26 36]]
Optimum gain: 36 for knapsck with maximum weight restriction: 13


In [20]:
###-- Printing the Knapsack value matrix as the slides ---------

from IPython.display import display, HTML

# create a Pandas DateFrame
df = pd.DataFrame(W, index = list( zip(items_value, items_weight)))

# from DataFrame to html code 
html = df.to_html()

# display the n-matrix nice
display(HTML(html))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
"(10, 4)",0,0,0,0,10,10,10,10,10,10,10,10,10,10
"(11, 4)",0,0,0,0,11,11,11,11,21,21,21,21,21,21
"(15, 5)",0,0,0,0,11,15,15,15,21,26,26,26,26,36


# Strings 

## Edit Distance (Distancia de edición)

### Introducción

* **(F)** Obtener la distancia de edición entre las cadenas
$x = [ G,C, G, T, A, T,   G, C, G, C, T, A,     A, C, G, C ]$ y  
$y = [ G,C,    T, A, T,   G, C, G, C, T, A, T,  A, C, G, C ]$
* **(F)** Obtener la distancia de edición entre las cadenas de caracteres "*The longest*" y "*longest day*"

<img src="edit_distance.png">


### La solución con programación dinámica

* Dadas las cadenas de caracteres $S$ y $T$ con longitud $M$ y $N$ respectivamente, consideremos las cadenas *prefijas*
\begin{align}
S_i &= [s_1, s_2, \ldots, s_i]  \\
T_j & = [t_1, t_2, \ldots, t_j] 
\end{align}

* Si $d_{i,j}$ es la distancia de edición entre $S_i$ y $T_j$ , queremos
encontrar $d_{M,N} = \text{dist}(S, T )$
* Observad que si $s_i = t_j$ , entonces $d_{i,j} = d_{i−1,j−1}$
* Y si $s_i \neq t_j$ tenemos tres opciones
 * Re-emplazar $t_j$ por $s_i$ ; entonces $d_{i,j} = 1 + d_{i−1,j−1}$
 * Eliminar $t_j$ de $T_j$ ; entonces $d_{i,j} = 1 + d_{i,j−1}$
 * Eliminar $s_i$ de $S_i$ ; entonces $d_{i,j} = 1 + d_{i−1,j}$
 
* Llegamos así a las siguiente ley de recurrencia para el problema de la distancia de edición

\begin{equation}
d_{i,j}=
\begin{cases}
d_{i−1,j−1}  & \text{si} &  s_i = t_j  \\
1 + \text{min} \left( d_{i−1,j−1} , d_{i,j−1} , d_{i−1,j} \right) 
& \text{si} & s_i \neq t_j
\end{cases}
\end{equation}

* Con las condiciones de contorno $d_{i,0} = i$ , $d_{0,j} = j$, donde
$d_{i,0} = \text{dist}(S_i ,\emptyset)$, $d_{0,j} =  \text{dist}(\emptyset, T_j)$,

In [4]:
def edDistDp(x:str, y:str)->ArrayLike:
    """ Calculate edit distance between sequences x and y using
    matrix dynamic programming. Return D matrix. """
    
    D = np.zeros((len(x)+1, len(y)+1), dtype=int)
    D[0, 1:] = range(1, len(y)+1)
    D[1:, 0] = range(1, len(x)+1)
    
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            # x[i] == y[j]
            if (x[i-1] == y[j-1]):
                D[i,j] = D[i-1, j-1]
                
            # x[i] != y[j]
            else :
                D[i, j] = min(D[i-1, j-1]+1, D[i-1, j]+1, D[i, j-1]+1)
    
    return D

# ----- Driver program ------
y = "biscuit"
x = "suitcase"

D = D = edDistDp(x, y)
print(D)
print (f'Edit distance:{D[-1,-1]}')

[[0 1 2 3 4 5 6 7]
 [1 1 2 2 3 4 5 6]
 [2 2 2 3 3 3 4 5]
 [3 3 2 3 4 4 3 4]
 [4 4 3 3 4 5 4 3]
 [5 5 4 4 3 4 5 4]
 [6 6 5 5 4 4 5 5]
 [7 7 6 5 5 5 5 6]
 [8 8 7 6 6 6 6 6]]
Edit distance:6


In [13]:
###---- Printing the edit distance matrix as the slides ---------

from IPython.display import display, HTML

# create a Pandas DateFrame

emptyset = '\u2205'

col = list(emptyset + y)
row = list(emptyset + x)

df = pd.DataFrame(D, columns = col, index = row) 

# from DataFrame to html code 
html = df.to_html()

# display the n-matrix nice
display(HTML(html))

,∅,b,i,s,c,u,i,t
∅,0,1,2,3,4,5,6,7
s,1,1,2,2,3,4,5,6
u,2,2,2,3,3,3,4,5
i,3,3,2,3,4,4,3,4
t,4,4,3,3,4,5,4,3
c,5,5,4,4,3,4,5,4
a,6,6,5,5,4,4,5,5
s,7,7,6,5,5,5,5,6
e,8,8,7,6,6,6,6,6


## Longest common subsequence

### Introducción

El problema de hallar la *subsecuencia* común más larga (**L**ongest **C**ommon **S**ubsequence, LCS) consiste en encontrar la subsecuencia más larga común a dos cadenas de caracteres dadas. 
Es un problema **diferente** al de encontrar la *subcadena* común más larga. 
A diferencia de las subcadenas, en las subsecuencias no se requiere que los caracteres que la forman ocupen posiciones consecutivas dentro de las cadenas originales. 


Por ejemplo, considere las cadenas de caracteres $S = [ABCD]$ y $T=[ACBAD]$. Encontramos las siguientes subsecuencias comunes:
* 5 subsecuencias comunes de longitud 2. A saber $[AB]$, $[AC]$, $[AD]$, $[BD]$ y $[CD]$.
* 2 subsecuencias comunes de longitud 3: $[ABD]$ y $[ACD]$.

Por tanto, $[ABD]$ y $[ACD]$ son las mayores subsecuencias comunes, LCS, a las cadenas $S$ y $T$ más largas.

El problema de hallar la subsecuencia común más larga es un problema clásico de informática. Constituye la base de programas de comparación de datos como la utilidad `diff` y tiene aplicaciones en lingüística computacional y bioinformática. También es ampliamente utilizado por los sistemas de control de revisión como Git para conciliar múltiples cambios realizados en una colección de archivos controlados por revisión o en herramientas para detectar plagio. 

Nos planteamos (i) un algoritmo para hallar la longitud de la LCS, en el ejemplo anterior es 3 y (ii) un algoritmo para hallar la LCS. 



### Longitud de la LCS: solución con programación dinámica
* Dadas las cadenas de caracteres $S$ y $T$ con longitud $M$ y $N$ respectivamente, consideremos las cadenas *prefijas* de tamaño $i$ y $j$
\begin{align}
S_i &= [s_1, s_2, \ldots, s_i]  \\
T_j & = [t_1, t_2, \ldots, t_j] 
\end{align}

* Sea $e_{i,j}$ la longitud de la LCS entre las cadenas $S_i$ y $T_j$. Puede ocurrir una de las siguientes dos posibilidades:
 * Si el último caracter de ambas cadenas es común, $s_i = t_j$, entonces deberíamos encontrar una LCS de las subcadenas $S_{i-1}$ y $T_{j-1}$ y añadir a dicha LCS el caracter $s_i = t_j$. Por tanto, la longitud de la LCS entre las cadenas  $S_{i}$ y $T_j$  se incrementaría, en una unidad respecto a la LCS ente las cadenas $S_{i-1}$ y $T_{j-1}$,
 $$
 e_{i,j} = 1 + e_{i-1, j-1}
 $$
  * Por el contrario si   $s_i \neq t_j$, deberemos resolver dos subproblemas: encontrar la LCS entre la cadena $S_{i-1}$ y $T_j$ y encontrar la LCS entre la cadena  $S_{i}$ y $T_{j-1}$. La mayor de las dos será la LCS entre las cadenas  $S_{i}$ y $T_j$ 
 $$ 
 e_{i, j} = \text{max} \left(  e_{i, j-1}, e_{i-1, j} \right)
 $$
 

* Como condiciones de contorno $e_{i,0} = 0$ y $e_{0,j} = 0$

* Llegamos así a la siguiente ley de recurrencia para el problema de hallar la longitud de la LCS entre las dos cadenas

\begin{equation}
e_{i,j}=
\begin{cases}
0  & \text{si} &  i = 0 \; \text{or} \;  j =0 \\  
1 + e_{i−1,j−1}  & \text{si} &  s_i = t_j  \\
\text{max} \left( e_{i,j−1} , e_{i−1,j} \right) 
& \text{si} & s_i \neq t_j
\end{cases}
\end{equation}


In [15]:
def LCS_length(x:str, y:str)->ArrayLike:
    """ Calculate  the LCS length between sequences x and y using
    matrix dynamic programming. Return matrix. """
    
    e = np.zeros((len(x)+1, len(y)+1), dtype=int)
    
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            # x[i] == y[j]
            if (x[i-1] == y[j-1]):
                e[i,j] = 1 + e[i-1, j-1]
                
            # x[i] != y[j]    
            else :
                e[i, j] = max(e[i-1, j], e[i, j-1])
    return e

#------ Driver programm
y = "BDCABA"
x = "ABCBDAB"

e = LCS_length (x, y)
print(e)
print(f'LCS length: {e[-1,-1]}')

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1]
 [0 1 1 1 1 2 2]
 [0 1 1 2 2 2 2]
 [0 1 1 2 2 3 3]
 [0 1 2 2 2 3 3]
 [0 1 2 2 3 3 4]
 [0 1 2 2 3 4 4]]
LCS length: 4


In [16]:
###---- Printing the LCS distance matrix as the slides ---------

from IPython.display import display, HTML

# create a Pandas DateFrame

emptyset = '\u2205'
col = list(emptyset + y)
row = list(emptyset + x)

df = pd.DataFrame(e, columns = col, index = row) 

# from DataFrame to html code 
html = df.to_html()

# display the n-matrix nice
display(HTML(html))

,∅,B,D,C,A,B,A
∅,0,0,0,0,0,0,0
A,0,0,0,0,1,1,1
B,0,1,1,1,1,2,2
C,0,1,1,2,2,2,2
B,0,1,1,2,2,3,3
D,0,1,2,2,2,3,3
A,0,1,2,2,3,3,4
B,0,1,2,2,3,4,4


### Obtener la LCS de dos cadenas  

*Ref: Cormen et al. 4 Edition p.397*

El elemento de matriz $e_{MN}$ hallado en la sección anterior nos proporciona la longitud de la $LCS$ entre las cadenas $T$ y $S$ de tamaño $M$ y $N$ respectivamente. Sin embargo únicamente con la matriz $e_{ij}$ no es posible obtener la $LCS$.
Definamos la matriz $b$, en cuyo elemento  $b_{ij}$ *almacenamos la procedencia* del valor asignado al elemento $e_{ij}$ (ver el código a continuación). Esto nos permitirá construir recursivamente la LCS, ver la función `LCS_print()`. 

In [51]:
def max_index (inputlist:List)->tuple:
    '''Return the mini and the index of the min'''

    #get the minimum value in the list
    max_value = max(inputlist)

    #get the index of minimum value 
    max_index=inputlist.index(max_value)
    return max_value, max_index


def LCS (x:str, y:str)->ArrayLike:
    """ Calculate  the LCS between sequences x and y using
    matrix dynamic programming. Return de matrix e and b"""
    
    e = np.zeros((len(x)+1, len(y)+1), dtype=int)
    b = np.empty((len(x)+1, len(y)+1), dtype=str)
    
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            # x[i] == y[j]
            if (x[i-1] == y[j-1]):
                e[i,j] = 1 + e[i-1, j-1]
                b[i,j] = 'D'
                
            # x[i] != y[j]    
            else :
                e[i, j], index = max_index ((e[i-1, j], e[i, j-1]))
                b[i, j] = 'L' if index else 'U'
    return e, b


def LCS_print (B:ArrayLike, x:str, i:int, j:int, ):
    #--Base case
    if i==0 or j==0:
        return
    
    #--General case
    # diagonal
    if B[i, j] == 'D':
        LCS_print (B, x, i-1, j-1)
        # elemento común de las cadenas
        print(x[i-1], end='')
    # Up
    elif B[i, j] == 'U':
        LCS_print (B, x, i-1, j)
    # Left
    elif B[i, j] == 'L':
        LCS_print (B, x, i, j-1)
        
# ---- Driver programm
y = "BDCABA"
x = "ABCBDAB"

e, b = LCS (x, y)

print(f'LCS length: {e[-1,-1]}')        

print('LCS:', end='')
LCS_print (b, x, len(x), len(y))

LCS length: 4
LCS:BCBA

## Multiplicación de matrices

### Introducción

Supongamos que tenemos cuatro matrices, $A, B, C$ y $D$, de dimensiones $A = 50  \times 10$, $B = 10 \times 40$, $C = 40 \times 30$ y 
$D = 30  \times 5$. 
Aunque la multiplicación de matrices no es conmutativa,
es asociativa, lo que significa que podemos utilizar paréntesis y evaluar el producto de matrices $ABCD$ de diferentes maneras. 
El algoritmo habitual para multiplicar dos matrices de dimensiones
$p \times q$ y $q \times r$, respectivamente, utiliza $p\cdot q \cdot r$ multiplicaciones escalares. (Utilizar un algoritmo teóricamente superior
como el algoritmo de Strassen no altera significativamente el problema que vamos a considerar, por lo que asumiremos este límite.)

¿Cuál es la mejor manera de realizar las tres multiplicaciones de matrices necesarias para calcular $ABCD$?
En el caso de cuatro matrices, es sencillo resolver el problema mediante una búsqueda exhaustiva, ya que solo hay cinco formas de *asociar* las matrices para realizar las multiplicaciones. Evaluamos cada caso a continuación:

* $(A) ( (BC) D)$: Evaluar el producto  $BC$ requiere $10 \times 40 \times 30 = 12 000$ multiplicaciones.
Evaluar $(BC)D$ requiere las 12,000 multiplicaciones para calcular $BC$, más  $10 \times 30 \times 5 = 1.500$ multiplicaciones, para un total de 13.500. 
Finalmente evaluar  $(A) ( (BC) D)$ requiere 13.500 multiplicaciones para $(BC)D$, más $50 \times 10 \times 
5 = 2.500$ multiplicaciones, para un total de 16.000 multiplicaciones.
*  $(A) (B (CD) ) $: Evaluar $CD$ requiere $40 \times 30 \times 5 = 6000$ multiplicaciones. Evaluar
$B(CD)$ requiere las 6000 multiplicaciones anteriores para calcular $CD$, más $10 \times 40 \times 5$ = 2000 multiplicaciones, para un total de 8000. Finalmente evaluar $(A) (B (CD) )$ requiere 8,000
multiplicaciones para $B(CD)$, más $50 \times 10 \times 5$ adicionales = 2500 multiplicaciones, para un total de 10.500 multiplicaciones.

* $(AB)(CD)$: Evaluar $(CD)$ requiere $40 \times 30 \times 5 = 6000$ multiplicaciones.
Evaluar $(AB)$ requiere $50 \times 10 \times 40 = 20,000$ multiplicaciones. 
Evaluar $(AB)(CD)$ requiere 6.000 multiplicaciones para $(CD)$, 20.000 multiplicaciones para $(AB)$, más $50 \times 40 \times 5 = 10,000$ multiplicaciones adicionales para un total de 36,000 multiplicaciones.

* $((AB) C)  (D)$: Evaluar $AB$ requiere $50 \times 10 \times 40 = 20,000$ multiplicaciones.
Evaluar $(AB)C$ requiere las 20 000 multiplicaciones para calcular $AB$, más
$50 \times 40 \times 30 = 60.000$ multiplicaciones adicionales, para un total de 80.000. 
Finalmente evaluar
$( (AB) C) (D)$ requiere 80.000 multiplicaciones para $(AB)C$, más $50 \times 30 \times 5 = 7500$ multiplicaciones, para un total de 87500 multiplicaciones.

* $(A(BC))  (D)$: Evaluar $BC$ requiere $10 \times 40 \times 30 = 12,000$ multiplicaciones.
Evaluar $A(BC)$ requiere las 12,000 multiplicaciones para calcular $BC$, más 
 $50 \times 10 \times 30 = 15.000$ multiplicaciones adicionales, para un total de 27.000. Evaluar $(A (BC) )$ (D) requiere 27,000 multiplicaciones para $A(BC)$, más $50 \times 30 \times 5 = 7500$ multiplicaciones adicionales, para un gran total de 34500 multiplicaciones.